In [ ]:
import pandas as pd
import time
import os
path = 'twitter/'
all_files = os.listdir(path)
count_files = len(os.listdir(path))
print count_files
print type(all_files)
print all_files[0]
print all_files[:3]

In [ ]:
limit = 280*40
print limit

In [ ]:
i=0
df = pd.DataFrame()
for filename in all_files:
    i=i+1
    if i <= limit:
        ##print filename
        new_path = "twitter"+"/" + str(filename)
        df0 = pd.read_json(new_path)
        df1 = df0.T
        df = df.append(df1)
print df.shape

In [ ]:
### df = pd.read_json('twitter/2015-10-07 22%3A16%3A58.283000tweets.json')
# all_tweets is a dataframe

In [ ]:
df.head(3)

In [ ]:
print len(df)
print type(df)
print df.dtypes.head(3)
print df.columns

In [ ]:
# reverse the columns and rows
all_tweets = df
print all_tweets.shape

In [ ]:
# Add and drop columns, reset index
all_tweets['category'] = ""
## all_tweets['user_from']= ""
all_tweets.reset_index(drop=True, inplace=True)
all_tweets.drop(['data_point','raw_source','tweet_id', 'user','user_location'],axis=1, inplace=True)

In [ ]:
all_tweets.head(3)

In [ ]:
prayer_dict = {'pray':'صلى','prayer':'صلاة','praying':'صلاة','muslim':'مسلم','Islam':'الإسلام',
               'mosque':'جامع','mosques':'جوامع','masjid':'مساجد','masjids':'مسجد',
               'call to prayer':'اذان','muezzin':'مؤذن','holy':'مقدس',
              'imam':'أئمة','imams':'امام','koran':'القرآن','chapter':'سور','verse':'آيات'}
food_dict = {'food':'طعام','eating':'يتناول الطعام','falafel':'فلافل','shawarma':'الشاورما',
            'starbucks':'ستاربكس','mcdonalds':'ماكدونالدز'}
fun_dict = {'restaurant':'مطاعم','restaurants':'مطعم','park':'حدائق','theater':'سينمات','museum':'متاحف',
           'bakery':'مخابز','gallery':'معرض','fair':'مهرجانات','zoo':'حديقة الحيوان','hotel':'فنادق',
           'starbucks':'ستاربكس','mcdonalds':'ماكدونالدز'}
parking_dict = {'stop':'يوقف','park':'وقف','parking lot':'موقف','parking':'مواقف','standing':'المواقف',
               'street parking':'يركن','garage':'يجرّش'}
traffic_dict = {'traffic':'مرور','underground':'سرا','crowd':'زحمة','crowds':'زحام','jammed':'اختناق',
                'erupted':'نشبت','reroute':'تحويلة','bump':'مطب','pothole':'مطبات'}
accident_dict = {'accident':'حادث','accidents':'حوادث','crashes':'حادثة','collision':'صدم','hit by':'صدمني',
                 'detour':'انعفطت',
                'wreck':'خربت','wreckage':'حطام'}
transit_dict = {'bus':'باص','taxi':'تاكسي','taxis':'تاكسيات','cab':'تكسي','limo':'ليموزين','fare':'أجرة',
                'cab fee':'اجرة','Careem':'كريم','Uber':'اوبر','driver':'سواق','chauffeur':'سائق','airport':'مطارات',
                'public transit':'مواصلات'}

In [ ]:
# Search for Arabic terms using unicode
# It works on English too

def count_search_terms(arabic_word):
    j=0
    search_term = unicode(arabic_word, encoding="utf-8")
    for row in all_tweets.itertuples():
        if search_term.lower() in row[1].lower():
            j+=1
    return j

In [ ]:
# Now count multiple terms at once
# The function counts both Arabic and English versions

def count_category(search_dict):
    i=0
    for key in search_dict:
        count = count_search_terms(search_dict[key])
        count = count + count_search_terms(key)
        i+=count
        print key + ": " + str(count)
    print "category total: " + str(i)

In [ ]:
count_category(traffic_dict)

In [ ]:
count_category(parking_dict)

In [ ]:
count_category(accident_dict)

In [ ]:
count_category(transit_dict)

In [ ]:
count_category(prayer_dict)

In [ ]:
count_category(fun_dict)

In [ ]:
# function decides if category is present
def has_category(tweet_content, search_dict):
    answer = False
#     tweet_content = unicode(tweet_content, encoding="utf-8")
    for english_word in search_dict.keys():
        if english_word.lower() in tweet_content.lower():
            answer = True
    for arabic_word in search_dict.values():
        arabic_word = unicode(arabic_word, encoding="utf-8")
        if arabic_word in tweet_content:
            answer = True
    return answer

In [ ]:
# function checks every row and adds categories
def fill_in_categories():
    for tweet in all_tweets.itertuples():
        idx = tweet[0]
        content = all_tweets.loc[idx,'content']
        if has_category(content, traffic_dict):
            all_tweets.loc[idx, 'category'] = 'traffic'
        elif has_category(content, transit_dict):
            all_tweets.loc[idx, 'category'] = 'transit'
        elif has_category(content, accident_dict):
            all_tweets.loc[idx, 'category'] = 'accident'
        elif has_category(content, parking_dict):
            all_tweets.loc[idx, 'category'] = 'parking'
        elif has_category(content, prayer_dict):
            all_tweets.loc[idx, 'category'] = 'prayer'
        elif has_category(content, fun_dict):
            all_tweets.loc[idx, 'category'] = 'fun'
        else:
            all_tweets.loc[idx, 'category'] = 'none'            

In [ ]:
fill_in_categories()

In [ ]:
all_tweets.head()

In [ ]:
def time_converter(input_string):
    ##orig_date = input_string[:-10] + input_string[-4:]
    ##convert = time.strptime(orig_date, '%a %b %d %H:%M:%S %Y')
    ##new_time = time.strftime("%A %I %p",convert)
    new_date = str(input_string[4:10])
    new_hour = str(input_string[11:13])
    return new_date, new_hour

In [ ]:
print time_converter('Tue Oct 13 17:08:31 +0000 2015')
print time_converter(all_tweets.loc[16,'time'])

In [ ]:
for row in all_tweets.itertuples():
    idx = int(row[0])
    new_time = time_converter(all_tweets.loc[idx,'time'])
    all_tweets.loc[idx,'Date'] = str(new_time[0])
    all_tweets.loc[idx,'Hour'] = float(new_time[1])

In [ ]:
all_tweets.head(5)

In [ ]:
all_tweets.to_csv('all_tweets.csv', index=False, encoding='utf-8')

In [ ]:
import pandas as pd
import numpy as np
np.sum(df['category'] == "transit")

In [ ]:
df = all_tweets
# Group the edges by their source
grouped_sources = df['Hour'].groupby(df['Date'])
## count and combine
edges_per_node = grouped_sources.count()
print edges_per_node

from collections import Counter
Counter(df['Date'])

In [ ]:
df_new = pd.DataFrame()
j=0
for i in dict.keys(Counter(df['Date'])):
    for j in range(0,24):
        ##print i + ", " + "Hour: " +str(j)
        count= np.sum((df['Date']==i) & (df['Hour']== j))
        ##print count
        df_new.loc[j,i] = float(count)
df_new.head(5)

In [ ]:
df_new.to_csv('df_new.csv', index=False, encoding='utf-8')

In [ ]:
## End

In [ ]:
all_tweets_categorized = all_tweets[all_tweets['category']!= 'none']
print len(all_tweets_categorized)

In [ ]:
all_tweets_categorized.reset_index(inplace=True, drop=True)
all_tweets_categorized.head()

In [ ]:
all_tweets_categorized.to_csv('all_tweets_categorized.csv', index=False, encoding='utf-8')